In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.display import Image
import imageio.v2
import csv

import tqdm as tqdm
import jax 
jax.config.update('jax_platform_name', 'cpu')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

from iLQR import iLQR, Path



In [2]:
def load_path(filepath: str):
    """
    Gets the centerline of the track from the trajectory data. We currently only
    support 2D track.

    Args:
        filepath (str): the path to file consisting of the centerline position.

    Returns:
        np.ndarray: centerline, of the shape (2, N).
    """
    x = []
    y = []
    with open(filepath) as f:
        spamreader = csv.reader(f, delimiter=',')
        for i, row in enumerate(spamreader):
            if i > 0:
                x.append(float(row[0]))
                y.append(float(row[1]))

    return np.array([x, y])

In [ ]:
x_cur = np.array([0, 5.4, 0, 3.14, 0])  # Initial state.

# Load ILQR
centerline = load_path('outerloop_center_smooth.csv')
path = Path(centerline, 0.6, 0.6, loop=True)

config_file = '/hdd/Git_Repo/PrincetonRaceCar/ROS_Core/src/Planning/ilqr_planning_ros/configs/ilqr.yaml'
solver = iLQR() #(config_file)
solver.update_path(path)

# define some parameters
n = 10
itr_receding = 30
init_control = np.zeros((2, n))
fig_prog_folder = 'fig/'

state_history = np.zeros((5,itr_receding))
t_process = np.zeros(itr_receding)
for i in (pbar := tqdm.tqdm(range(itr_receding))):
    
    plan = solver.plan(x_cur, init_control)
    states = plan['states']
    controls = plan['controls']
    x_cur = states[:,1]
    state_history[:,i]  = x_cur
    init_control[:,:-1] = controls[:,1:]
    pbar.set_description(f"iLQR takes : {plan['t_process']:.2f} sec']")
    t_process[i] = plan['t_process']
    # plot
    plt.clf()
    path.plot_track()
    sc = plt.scatter(
        state_history[0, :i], state_history[1, :i], s=24,
        c=state_history[2, :i], cmap=cm.jet, vmin=0, vmax=5,
        edgecolor='none', marker='o'
    )
    cbar = plt.colorbar(sc)
    cbar.set_label(r"velocity [$m/s$]", size=20)
    
    plt.plot(states[0, :], states[1, :], 'r', linewidth=2)
    circle1 = plt.Circle((-2, 5.), 0.35, color='r')
    plt.gca().add_patch(circle1)
    plt.axis('equal')
    plt.savefig(os.path.join(fig_prog_folder, str(i)+".png"), dpi=200)
plt.close('All')

gif_path = os.path.join(fig_prog_folder, 'rollout.gif')
with imageio.get_writer(gif_path, mode='I') as writer:
    for i in range(itr_receding):
        filename = os.path.join(fig_prog_folder, str(i)+".png")
        image = imageio.imread(filename)
        writer.append_data(image)

plt.figure()
plt.plot(state_history[2,:])
plt.title('Velocity')


plt.figure()
a = state_history[2, :]**2 / 0.257 * np.tan(state_history[-1, :])
plt.plot(a)

plt.figure()
plt.plot(t_process)




In [6]:
centerline = load_path('outerloop_center_smooth.csv')
path = Path(centerline, 0.6, 0.6, loop=True)
solver = iLQR()
print(solver.backward_pass._cache_size())

iLQR config: {'num_dim_x': 5, 'num_dim_u': 2, 'n': 10, 'dt': 0.1, 'max_iter': 100, 'tol': 0.001, 'line_search_a': 0.1, 'line_search_b': 3, 'line_search_c': 1, 'reg_min': 0.001, 'reg_max': 100000.0, 'reg_scale_down': 0.5, 'reg_scale_up': 5.0, 'reg_init': 1, 'wheelbase': 0.257, 'radius': 0.13, 'delta_max': 0.35, 'delta_min': -0.35, 'v_max': 5.0, 'v_min': 0.0, 'omega_min': -6.0, 'omega_max': 6.0, 'a_max': 5.0, 'a_min': -5.0, 'dim_closest_pt_x': 0, 'dim_closest_pt_y': 1, 'dim_path_slope': 2, 'path_cost_type': 'quadratic', 'path_weight': 10.0, 'path_huber_delta': 2, 'dim_vel_ref': 3, 'vel_cost_type': 'quadratic', 'vel_weight': 2.0, 'vel_huber_delta': 0.5, 'lat_accel_thres': 6.0, 'lat_accel_a': 5.0, 'lat_accel_b': 1.0, 'dim_progress': 4, 'progress_weight': 1.0, 'ctrl_cost_type': 'quadratic', 'ctrl_cost_accel_weight': 1, 'ctrl_cost_steer_weight': 1, 'ctrl_cost_accel_huber_delta': 1.0, 'ctrl_cost_steer_huber_delta': 1.0, 'dim_obs_x': 5, 'dim_obs_y': 6, 'dim_obs_radius': 7, 'obs_a': 10.0, 'obs_

In [ ]:
solver.update_path(path)

x_cur = np.array([1.5, 5.4, 1, 3.14, 0])
plan = solver.plan(x_cur)

path.plot_track()
path.plot_track_center()
plt.plot(plan['states'][0, :], plan['states'][1, :], 'r-*', linewidth=2)

circle1 = plt.Circle((0, 5.4), 0.35, color='r')
plt.gca().add_patch(circle1)
plt.axis('equal')

print(plan['status'])

plt.figure()
a = plan['states'][2, :] 
plt.plot(a)
plt.title('Velocity')

plt.figure()
plt.plot(plan['controls'][0,:])
plt.plot(plan['controls'][-1,:], '--')
plt.title('Steering')

plt.figure()
a = plan['states'][2, :]**2 / 0.257 * np.tan(plan['states'][-1, :])
plt.plot(a)
plt.title('Lateral Acceleration')


In [ ]:
print(plan['t_process'])
